In [ ]:
"""
condo_risk_scoring_v3.py

เวอร์ชัน 3:
- ทุกอย่างจาก v2
- เพิ่มการคำนวณ percentile ของแต่ละคอนโดเทียบกับคอนโดอื่นใน "district" เดียวกัน
  (overall_score_0_100, safety_score_0_100, price_score_0_100, value_index)
- บันทึกไฟล์ output แยกไฟล์ตาม district (ใน ../Data/score) และไฟล์รวมทั้งหมด
"""

import os
import re
from typing import Dict, Any

import numpy as np
import pandas as pd
import geopandas as gpd


# ===================== CONFIG =====================

TARGET_DISTRICTS_THAI = [
    "จตุจักร",
    "ประเวศ",
    "วัฒนา",
    "บางกะปิ",
    "คลองเตย",
    "บางแค",
    "ปทุมวัน",
    "บางเขน",
]

INPUT_DIR = "../Data/clean"
OUTPUT_DIR = "../Data/score"

CONDO_FILES = [
    f"{INPUT_DIR}/condo_จตุจักร_clean.csv",
    f"{INPUT_DIR}/condo_ประเวศ_clean.csv",
    f"{INPUT_DIR}/condo_วัฒนา_clean.csv",
    f"{INPUT_DIR}/condo_บางกะปิ_clean.csv",
    f"{INPUT_DIR}/condo_คลองเตย_clean.csv",
    f"{INPUT_DIR}/condo_บางแค_clean.csv",
    f"{INPUT_DIR}/condo_ปทุมวัน_clean.csv",
    f"{INPUT_DIR}/condo_บางเขน_clean.csv",
]

CONDO_LAT_COL = "latitude"
CONDO_LON_COL = "longitude"
CONDO_DISTRICT_COL = "district_name"
CONDO_NAME_COL = "condo_name"

CONDO_PRICE_COL = "Price"
CONDO_SIZE_COL = "Room_Size"

TRAFFY_FILE = f"{INPUT_DIR}/bangkok_traffy_clean.csv"

BTS_FILE = f"{INPUT_DIR}/bts_station.csv"
BTS_LAT_COL = "lat"
BTS_LON_COL = "lng"
BTS_NAME_COL = "name"

CONDO_BUFFER_RADIUS_M = 1000
BTS_NEAR_RADIUS_M = 500
USER_TRAVEL_MODE = "bts"


# 9) การแมป type ของ Traffy → category
TYPE_TO_CATEGORY: Dict[str, str] = {
    # ---- ROAD / TRAVEL ----
    "ถนน": "road",
    "ทางเท้า": "road",
    "จราจร": "road",
    "การเดินทาง": "road",
    "สะพาน": "road",
    "ป้ายจราจร": "road",
    "ป้าย": "road",

    # ---- FLOOD / DRAINAGE ----
    "น้ำท่วม": "flood",
    

    # ---- SAFETY / SECURITY ----
    "ความปลอดภัย": "safety",
    "แสงสว่าง": "safety",
    "สายไฟ": "safety",
    "สัตว์จรจัด": "safety",
    "คนจรจัด": "safety",

    # ---- NOISE ----
    "เสียงรบกวน": "noise",

    # ---- CLEANLINESS / OBSTRUCTION ----
    "ความสะอาด": "cleanliness",
    "กีดขวาง": "cleanliness",
    "ห้องน้ำ": "cleanliness",

    # ---- ENVIRONMENT / AIR ----
    "ต้นไม้": "environment",
    "PM2.5": "environment",
    "ท่อระบายน้ำ": "flood",
    "คลอง": "flood",

    # ---- META / OTHER ----
    "ร้องเรียน": "other",
    "สอบถาม": "other",
    "เสนอแนะ": "other",
    "อื่นๆ": "other",
}

DEFAULT_CATEGORY = "other"

# 10) น้ำหนักของแต่ละ category (ยิ่งสูง = ยิ่งร้ายแรง)
CATEGORY_WEIGHTS: Dict[str, float] = {
    "flood": 4.0,
    "safety": 3.5,
    "noise": 3.0,
    "road": 2.5,
    "cleanliness": 2.5,
    "environment": 3.0,
    "other": 1.0,
}

RESOLVED_STATES = ["DONE", "Done", "done", "เสร็จสิ้น"]
REOPEN_PENALTY_PER_TIME = 0.3
REOPEN_PENALTY_MAX_TIMES = 3


# ===================== UTIL =====================

def clean_district(name: Any) -> str | None:
    if pd.isna(name):
        return None
    s = str(name).strip()
    if s.startswith("เขต"):
        s = s[3:].strip()
    return s


def split_type_list(raw: Any) -> list[str]:
    if pd.isna(raw):
        return []
    s = str(raw).strip()
    if s.startswith("(") and s.endswith(")"):
        s = s[1:-1]
    if not s:
        return []
    parts = [p.strip() for p in s.split(",") if p.strip()]
    return parts


def map_type_to_category(t: Any) -> str:
    if pd.isna(t):
        return DEFAULT_CATEGORY
    s = str(t).strip()
    return TYPE_TO_CATEGORY.get(s, DEFAULT_CATEGORY)


def compute_resolution_factor(state: Any, count_reopen: Any) -> float:
    state_str = str(state).strip()
    try:
        reopen = int(count_reopen)
    except (ValueError, TypeError):
        reopen = 0
    base = 1.0 if state_str in RESOLVED_STATES else 1.5
    penalty = REOPEN_PENALTY_PER_TIME * min(reopen, REOPEN_PENALTY_MAX_TIMES)
    return base + penalty


def get_travel_factor(mode: str, near_bts: bool) -> float:
    mode = mode.lower()
    if mode == "bts":
        return 0.0 if near_bts else 1.0
    if mode == "mixed":
        return 0.5 if near_bts else 1.0
    return 1.0


def classify_price_risk(row, price_th, risk_th) -> str:
    price_score = row["price_score_0_100"]
    risk_score = row["risk_score_0_100"]
    expensive = price_score > price_th
    cheap = price_score < price_th
    low_risk = risk_score < risk_th
    high_risk = risk_score > risk_th
    if not (expensive or cheap):
        if low_risk:
            return "Good"
        elif high_risk:
            return "Okay"
        else:
            return "Okay"
    if expensive and low_risk:
        return "Perfect"
    if expensive and not low_risk:
        return "Bad"
    if cheap and low_risk:
        return "Good"
    return "Okay"


def parse_price(x: Any) -> float | np.ndarray:
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = re.sub(r"[^0-9.,]", "", s)
    s = s.replace(",", "")
    if s == "":
        return np.nan
    try:
        return float(s)
    except ValueError:
        return np.nan


def parse_room_size(x: Any) -> float | np.ndarray:
    if pd.isna(x):
        return np.nan
    s = str(x)
    m = re.search(r"([0-9]+(\.[0-9]+)?)", s)
    if not m:
        return np.nan
    try:
        return float(m.group(1))
    except ValueError:
        return np.nan


# def safe_filename(s: str) -> str:
#     return re.sub(r"[^\w\-_. ]", "_", str(s)).replace(" ", "_")


# ===================== MAIN =====================

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # LOAD condos
    condo_list = []
    for f in CONDO_FILES:
        df = pd.read_csv(f)
        condo_list.append(df)
    condos = pd.concat(condo_list, ignore_index=True)

    condos["district_clean"] = condos[CONDO_DISTRICT_COL].apply(clean_district)
    condos = condos[condos["district_clean"].isin(TARGET_DISTRICTS_THAI)].copy()
    condos = condos.dropna(subset=[CONDO_LAT_COL, CONDO_LON_COL])
    condos["condo_id"] = range(len(condos))

    # LOAD traffy
    traffy = pd.read_csv(TRAFFY_FILE)
    traffy["district_clean"] = traffy["district"].apply(clean_district)
    traffy = traffy[traffy["district_clean"].isin(TARGET_DISTRICTS_THAI)].copy()
    traffy = traffy.dropna(subset=["lat", "lon"])

    traffy["type_list"] = traffy["type"].apply(split_type_list)
    traffy_long = traffy.explode("type_list").rename(columns={"type_list": "type_clean"})
    traffy_long["type_clean"] = traffy_long["type_clean"].fillna("อื่นๆ")
    traffy_long["category"] = traffy_long["type_clean"].apply(map_type_to_category)

    # Geo
    condos_gdf = gpd.GeoDataFrame(
        condos,
        geometry=gpd.points_from_xy(condos[CONDO_LON_COL], condos[CONDO_LAT_COL]),
        crs="EPSG:4326",
    )
    traffy_gdf = gpd.GeoDataFrame(
        traffy_long,
        geometry=gpd.points_from_xy(traffy_long["lon"], traffy_long["lat"]),
        crs="EPSG:4326",
    )
    condos_gdf = condos_gdf.to_crs(epsg=3857)
    traffy_gdf = traffy_gdf.to_crs(epsg=3857)

    # buffer & join
    condo_buffer = condos_gdf.copy()
    condo_buffer["geometry"] = condo_buffer.geometry.buffer(CONDO_BUFFER_RADIUS_M)
    condo_area = condo_buffer[["condo_id", CONDO_NAME_COL, "district_clean", "geometry"]].copy()

    joined = gpd.sjoin(
        traffy_gdf,
        condo_area,
        predicate="within",
        how="inner",
    )

    # ticket scoring
    joined["category_weight"] = joined["category"].map(CATEGORY_WEIGHTS).fillna(CATEGORY_WEIGHTS.get(DEFAULT_CATEGORY, 1.0))
    joined["resolution_factor"] = joined.apply(lambda row: compute_resolution_factor(row["state"], row["count_reopen"]), axis=1)
    joined["ticket_score"] = joined["category_weight"] * joined["resolution_factor"]
    joined["is_unresolved"] = ~joined["state"].isin(RESOLVED_STATES)

    # unique ticket counts (per condo) — ไม่ใช่หลัง explode
    unique_tickets = joined.groupby("condo_id")["ticket_id"].nunique().reset_index().rename(columns={"ticket_id": "unique_tickets"})
    unique_unresolved = joined[joined["is_unresolved"] == True].groupby("condo_id")["ticket_id"].nunique().reset_index().rename(columns={"ticket_id": "unique_unresolved"})
    condo_uniques = unique_tickets.merge(unique_unresolved, on="condo_id", how="left")

    # aggregate per condo & category (after explode)
    per_condo_cat = joined.groupby(["condo_id", "category"], as_index=False).agg(
        problems_count=("ticket_id", "count"),
        total_score=("ticket_score", "sum"),
        unresolved_count=("is_unresolved", "sum"),
    )

    condo_cat_counts = per_condo_cat.pivot(index="condo_id", columns="category", values="problems_count").fillna(0)
    condo_cat_counts.columns = [f"count_{c}" for c in condo_cat_counts.columns]
    condo_cat_counts = condo_cat_counts.reset_index()

    condo_cat_scores = per_condo_cat.pivot(index="condo_id", columns="category", values="total_score").fillna(0)
    condo_cat_scores.columns = [f"score_{c}" for c in condo_cat_scores.columns]
    condo_cat_scores = condo_cat_scores.reset_index()

    condo_cat_unresolved = per_condo_cat.pivot(index="condo_id", columns="category", values="unresolved_count").fillna(0)
    condo_cat_unresolved.columns = [f"unresolved_{c}" for c in condo_cat_unresolved.columns]
    condo_cat_unresolved = condo_cat_unresolved.reset_index()

    condo_totals = per_condo_cat.groupby("condo_id", as_index=False).agg(
        total_problems=("problems_count", "sum"),
        total_score=("total_score", "sum"),
        total_unresolved=("unresolved_count", "sum"),
    )

    # merge features
    condos_features = condos_gdf.merge(condo_totals, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_uniques, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_counts, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_scores, on="condo_id", how="left")
    condos_features = condos_features.merge(condo_cat_unresolved, on="condo_id", how="left")

    feature_cols = [c for c in condos_features.columns if c.startswith("count_") or c.startswith("score_") or c.startswith("unresolved_") or c in ["total_problems", "total_score", "total_unresolved", "unique_tickets", "unique_unresolved"]]
    condos_features[feature_cols] = condos_features[feature_cols].fillna(0)

    # BTS nearest
    bts = pd.read_csv(BTS_FILE)
    bts_gdf = gpd.GeoDataFrame(bts, geometry=gpd.points_from_xy(bts[BTS_LON_COL], bts[BTS_LAT_COL]), crs="EPSG:4326").to_crs(epsg=3857)

    nearest = gpd.sjoin_nearest(condos_features, bts_gdf[[BTS_NAME_COL, "geometry"]], how="left", distance_col="dist_to_bts_m")
    condos_features["dist_to_bts_m"] = nearest["dist_to_bts_m"]
    condos_features["nearest_bts"] = nearest[BTS_NAME_COL]
    condos_features["near_bts"] = condos_features["dist_to_bts_m"] <= BTS_NEAR_RADIUS_M

    # compute risk (normalize per category -> weight)
    category_to_count_col = {}
    for cat in CATEGORY_WEIGHTS.keys() | {DEFAULT_CATEGORY}:
        col_name = f"count_{cat}"
        if col_name in condos_features.columns:
            category_to_count_col[cat] = col_name

    risk_components_cols = []
    for cat, count_col in category_to_count_col.items():
        max_val = condos_features[count_col].max()
        norm_col = f"norm_{cat}"
        if max_val > 0:
            condos_features[norm_col] = condos_features[count_col] / max_val
        else:
            condos_features[norm_col] = 0.0
        w = CATEGORY_WEIGHTS.get(cat, CATEGORY_WEIGHTS.get(DEFAULT_CATEGORY, 1.0))
        risk_col = f"risk_{cat}"
        condos_features[risk_col] = condos_features[norm_col] * w
        risk_components_cols.append(risk_col)

    road_risk_col = "risk_road"
    non_travel_risk_cols = [c for c in risk_components_cols if c != road_risk_col]
    condos_features["risk_non_travel"] = condos_features[non_travel_risk_cols].sum(axis=1) if non_travel_risk_cols else 0.0
    condos_features["risk_travel_raw"] = condos_features[road_risk_col] if road_risk_col in condos_features.columns else 0.0
    condos_features["travel_factor"] = condos_features["near_bts"].apply(lambda nb: get_travel_factor(USER_TRAVEL_MODE, bool(nb)))
    condos_features["risk_travel_effective"] = condos_features["risk_travel_raw"] * condos_features["travel_factor"]
    condos_features["raw_risk"] = condos_features["risk_non_travel"] + condos_features["risk_travel_effective"]

    raw_min = condos_features["raw_risk"].min()
    raw_max = condos_features["raw_risk"].max()
    condos_features["risk_score_0_100"] = ((condos_features["raw_risk"] - raw_min) / (raw_max - raw_min) * 100) if raw_max > raw_min else 0.0
    condos_features["safety_score_0_100"] = 100 - condos_features["risk_score_0_100"]

    # price parsing & compute price per sqm
    condos_features[CONDO_PRICE_COL] = condos_features[CONDO_PRICE_COL].apply(parse_price)
    condos_features[CONDO_SIZE_COL] = condos_features[CONDO_SIZE_COL].apply(parse_room_size)
    condos_features["price_per_sqm"] = condos_features[CONDO_PRICE_COL] / condos_features[CONDO_SIZE_COL]
    condos_features.loc[(condos_features["price_per_sqm"] <= 0) | (~np.isfinite(condos_features["price_per_sqm"])), "price_per_sqm"] = np.nan

    min_p = condos_features["price_per_sqm"].min()
    max_p = condos_features["price_per_sqm"].max()
    if pd.notna(min_p) and pd.notna(max_p) and max_p > min_p:
        condos_features["price_score_0_100"] = (condos_features["price_per_sqm"] - min_p) / (max_p - min_p) * 100
    else:
        condos_features["price_score_0_100"] = 50.0
    condos_features["price_score_0_100"] = condos_features["price_score_0_100"].fillna(50.0)

    price_th = condos_features["price_score_0_100"].median()
    risk_th = condos_features["risk_score_0_100"].median()
    condos_features["quality_label"] = condos_features.apply(lambda row: classify_price_risk(row, price_th, risk_th), axis=1)
    condos_features["value_index"] = (100 - condos_features["risk_score_0_100"]) - condos_features["price_score_0_100"]

    # overall score
    affordability = 100 - condos_features["price_score_0_100"]
    condos_features["overall_score_0_100"] = 0.7 * condos_features["safety_score_0_100"] + 0.3 * affordability

    # ========== NEW: percentiles IN DISTRICT ==========
    # For each district, compute percentile rank (0-100) of certain scores:
    pct_cols = {
        "overall_score_0_100": "overall_pct_in_district",
        "safety_score_0_100": "safety_pct_in_district",
        "price_score_0_100": "price_pct_in_district",
        "value_index": "value_index_pct_in_district",
    }

    # Use groupby + rank(pct=True) to compute percentile within district
    for score_col, out_col in pct_cols.items():
        # default 0 if district has single item or NaN
        condos_features[out_col] = condos_features.groupby("district_clean")[score_col].transform(
            lambda x: x.rank(method="average", pct=True) * 100 if x.notna().sum() > 0 else 0
        )
        # fill NaN with 0
        condos_features[out_col] = condos_features[out_col].fillna(0)

    # ========== SAVE overall + per-district files ==========
    output_all = f"{OUTPUT_DIR}/condos_scored_v3_all.csv"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    # drop geometry column before saving overall CSV
    output_df = condos_features.drop(columns="geometry")
    output_df.to_csv(output_all, index=False)
    print(f"Saved overall file: {output_all}")

    # --- Reorder columns: put overall_pct_in_district then overall_score_0_100 at the far right ---
    # drop geometry if present
    if "geometry" in output_df.columns:
        output_df = output_df.drop(columns=["geometry"])

    # desired right-most cols (adjust names if you used other labels)
    right_cols = ["overall_pct_in_district", "overall_score_0_100"]

    # ensure they exist (avoid KeyError)
    right_cols = [c for c in right_cols if c in output_df.columns]

    # build new order: all current cols except right_cols, then append right_cols in order
    other_cols = [c for c in output_df.columns if c not in right_cols]
    new_col_order = other_cols + right_cols

    # reorder dataframe
    output_df = output_df.loc[:, new_col_order]

    # now save the overall file
    final_columns = [
    'Project_Name', 'condo_name', 'Price', 'price_per_sqm',
    'Room_Type', 'Floor', 'Bedrooms', 'Bathrooms', 'Room_Size',
    'Original_Link', 'district_clean', 'latitude', 'longitude',
    'price_score_0_100', 'quality_label', 'overall_pct_in_district',
    'overall_score_0_100']
    output_df = output_df[final_columns]
    output_all = f"{OUTPUT_DIR}/condos_scored_all.csv"
    output_df.to_csv(output_all, index=False)
    print(f"Saved overall file (with reordered columns): {output_all}")

    # save per-district files with same column order
    districts = output_df["district_clean"].fillna("unknown").unique()
    for d in districts:
        if pd.isna(d):
            continue
        df_d = output_df[output_df["district_clean"] == d].copy()
        fname = f"{OUTPUT_DIR}/{d}_condos_scored.csv"
        df_d = df_d[final_columns]
        df_d.to_csv(fname, index=False)
        print("Saved per-district file:", fname)

    # save per-district files
    districts = condos_features["district_clean"].fillna("unknown").unique()
    saved_files = []
    for d in districts:
        if pd.isna(d):
            continue
        df_d = output_df[output_df["district_clean"] == d].copy()
        fname = f"{OUTPUT_DIR}/{d}_condos_scored.csv"
        df_d = df_d[final_columns]
        df_d.to_csv(fname, index=False)
        saved_files.append(fname)

    print("Saved per-district files:")
    for p in saved_files:
        print(" -", p)

    # sample head
    print(output_df[final_columns].head(15).to_string(index=False))


if __name__ == "__main__":
    main()


Saved overall file: ../Data/score/condos_scored_v3_all.csv
Saved overall file (with reordered columns): ../Data/score/condos_scored_v3_all.csv
Saved per-district file: ../Data/score/จตุจักร_condos_scored.csv
Saved per-district file: ../Data/score/ประเวศ_condos_scored.csv
Saved per-district file: ../Data/score/วัฒนา_condos_scored.csv
Saved per-district file: ../Data/score/บางกะปิ_condos_scored.csv
Saved per-district file: ../Data/score/คลองเตย_condos_scored.csv
Saved per-district file: ../Data/score/บางแค_condos_scored.csv
Saved per-district file: ../Data/score/ปทุมวัน_condos_scored.csv
Saved per-district file: ../Data/score/บางเขน_condos_scored.csv
Saved per-district files:
 - ../Data/score/จตุจักร_condos_scored.csv
 - ../Data/score/ประเวศ_condos_scored.csv
 - ../Data/score/วัฒนา_condos_scored.csv
 - ../Data/score/บางกะปิ_condos_scored.csv
 - ../Data/score/คลองเตย_condos_scored.csv
 - ../Data/score/บางแค_condos_scored.csv
 - ../Data/score/ปทุมวัน_condos_scored.csv
 - ../Data/score/บางเ